In [1]:
from keras import models
from keras.layers.core import Activation, Reshape, Permute
from keras.layers.convolutional import Convolution2D, MaxPooling2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint

from unet_project.image_utils import ImageUtils
from unet_project.data_augmentation import DataAugmentation
from unet_project.segnet import SegNet
from unet_project.create_patches import PatchesCreator
import cv2
import numpy as np
from tqdm import tqdm_notebook

Using TensorFlow backend.


In [2]:
path_to_imgs = '/home/ajuska/Dokumenty/Skola/diplomka/disk_data/imgs/'
path_to_masks = '/home/ajuska/Dokumenty/Skola/diplomka/disk_data/masks/'
img_height = 320
img_width = 320
n_labels = 1
kernel = 3
img_channels = 1

In [3]:
image_utils = ImageUtils(path_to_imgs, path_to_masks, img_height, img_width)
imgs_masks = image_utils.get_preprocessed_images()

# data_augmentor = DataAugmentation(imgs_masks, how_many=1)
# aug_imgs, aug_masks = data_augmentor.extend_database()

imgs = []
masks = []
for key, val in tqdm_notebook(imgs_masks.items()):
    imgs.append(val[0])
    masks.append(val[1])

trainX = [cv2.resize(x, (img_height, img_width)) for x in imgs]
trainY = [cv2.resize(x, (img_height, img_width)) for x in masks]

  2%|▏         | 3/186 [00:00<00:06, 26.99it/s]

Loading images.


100%|██████████| 186/186 [00:03<00:00, 59.27it/s]



Loading masks.


100%|██████████| 186/186 [00:00<00:00, 349056.17it/s]


Preprocessing images.


In [4]:
trainX = np.array([cv2.cvtColor(x.astype(np.uint8), cv2.COLOR_BGR2GRAY) for x in trainX])
trainX = np.array([np.expand_dims(x/255, axis=-1) for x in trainX])

In [5]:
trainY = np.reshape(trainY,(len(trainY),img_height*img_width,img_channels))

In [5]:
# # trainX = np.array([np.expand_dims(x/255, axis=-1) for x in trainX])
# trainX = np.array([np.expand_dims(x/255, axis=-1) for x in trainX])

# # trainX = np.array([x/255 for x in trainX])
# trainY = np.array([x/255 for x in trainY])

In [6]:
trainX.shape

(186, 320, 320, 1)

In [7]:
trainY.shape

(186, 102400, 1)

In [8]:
segnet = SegNet(img_height=img_height, img_width=img_width, img_channels=img_channels, 
                number_labels=n_labels, kernel_size=kernel)
model = segnet.create_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 320, 320, 64)      640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 320, 320, 64)      256       
_________________________________________________________________
activation_1 (Activation)    (None, 320, 320, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 320, 320, 64)      36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 320, 320, 64)      256       
_________________________________________________________________
activation_2 (Activation)    (None, 320, 320, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 160, 160, 64)      0         
__________

In [ ]:
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint('model-test.h5', verbose=1, save_best_only=True)
results = model.fit(trainX, trainY, validation_split=0.1, batch_size=32, epochs=50, verbose=1,
                    callbacks=[earlystopper, checkpointer])

Train on 167 samples, validate on 19 samples
Epoch 1/50


In [ ]:
history = autoencoder.fit(train_data, train_label, batch_size=batch_size, nb_epoch=nb_epoch, verbose=1)

In [ ]:
with open('model_5l_segnet.json', 'w') as outfile:
    outfile.write(json.dumps(json.loads(autoencoder.to_json()), indent=2))

In [11]:
with open('model_5l_segnet.json') as model_file:
    autoencoder = models.model_from_json(model_file.read())

In [13]:
from keras import models
from keras.optimizers import SGD

In [14]:
optimizer = SGD(lr=0.001, momentum=0.9, decay=0.0005, nesterov=False)
autoencoder.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])
print('Compiled: OK')

Compiled: OK
